# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center>SQL -2<center></span>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Contenido</span>

In [56]:
SHOW DATABASES;

Database GRUPOS birdwatchers information_schema mysql performance_schema rookery test

## <span style="color:blue">Primera base de datos</span>

In [57]:
CREATE DATABASE GRUPOS_COLOMBIA
CHARACTER SET 'utf8mb4'
COLLATE 'utf8mb4_general_ci';

In [ ]:
SHOW DATABASES;

In [ ]:
USE GRUPOS_COLOMBIA;

## Creación de tablas

### PAIS

In [12]:
CREATE TABLE PAIS(
IdPai MEDIUMINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
NomPai VARCHAR(255) UNIQUE NOT NULL,
AbrevPai VARCHAR(3) UNIQUE
)ENGINE = InnoDB;

In [ ]:
CREATE TABLE EN_PAIS(
AbrevPai VARCHAR(3) UNIQUE NOT NULL,
NomPai VARCHAR(255) UNIQUE NOT NULL
)ENGINE = InnoDB;

In [ ]:
LOAD DATA
LOCAL INFILE '/home/alvaro/Documents/Cursos/Big_Data/Datos/EN_PAIS.csv'
INTO TABLE EN_PAIS
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

In [ ]:
INSERT INTO PAIS(NomPai, AbrevPai)
SELECT NomPai, AbrevPai FROM EN_PAIS;


In [ ]:
SELECT COUNT(*) FROM PAIS;
# 174

### DEPARTAMENTO

In [ ]:
CREATE TABLE DEPARTAMENTO(
  IdDep MEDIUMINT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
  NomDep VARCHAR(100) NOT NULL,
  IdPai MEDIUMINT UNSIGNED NOT NULL,
  CONSTRAINT
    fk_DEPARTAMENTO_PAIS FOREIGN KEY (IdPai) REFERENCES PAIS (IdPai)
    ON DELETE CASCADE
    ON UPDATE RESTRICT
) ENGINE = InnoDB;

In [ ]:
CREATE TABLE EN_DEPARTAMENTO(
AbrevPai VARCHAR(3)  NOT NULL,
Abrev_Dep VARCHAR(3)  NOT NULL,
Abrev_Reg VARCHAR(3)  NOT NULL,    
NomDep VARCHAR(255)  NOT NULL,
NorDep VARCHAR(255)  NOT NULL
)ENGINE = InnoDB;

In [ ]:
LOAD DATA
LOCAL INFILE '/home/alvaro/Documents/Cursos/Big_Data/Datos/EN_DEPARTAMENTO.csv'
INTO TABLE EN_DEPARTAMENTO
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

### MODIFICA ALGUNOS REGISTROS

In [ ]:
UPDATE EN_DEPARTAMENTO
SET NomDep = "NA"
WHERE NomDep = "No Aplica";


### VERIFICA

In [ ]:
SELECT T.AbrevPai, S.NomDep, COUNT(S.NomDep) AS FRECUENCIA 
FROM EN_DEPARTAMENTO  AS S, EN_DEPARTAMENTO  AS T
WHERE T.NomDep = S.NomDep AND T.AbrevPai = S.AbrevPai
GROUP BY S.NomDep, T.AbrevPai
HAVING FRECUENCIA > 0
ORDER BY NomDep ASC, AbrevPai ASC;

## Sube datos

In [ ]:
INSERT INTO DEPARTAMENTO(NomDep, IdPai)
SELECT D.NomDep, P.IdPai
FROM PAIS AS P 
INNER JOIN EN_DEPARTAMENTO AS D
ON P.AbrevPai = D.AbrevPai;

### Verificacion

In [ ]:
SELECT NomDep, NomPai 
FROM DEPARTAMENTO AS D, PAIS AS P
WHERE D.idPai = P.IdPai;

### MUNICIPIO

In [ ]:
CREATE TABLE MUNICIPIO(
  IdMun MEDIUMINT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
  NomMun VARCHAR(100) NOT NULL,
  IdDep MEDIUMINT UNSIGNED  NOT NULL,
  CONSTRAINT
    fk_MUNICIPIO_DEPARTAMENTO FOREIGN KEY (IdDep) REFERENCES DEPARTAMENTO (IdDep)
    ON DELETE CASCADE
    ON UPDATE RESTRICT
) ENGINE = InnoDB;

In [ ]:
CREATE TABLE EN_MUNICIPIO(
  CodRh    VARCHAR(100),
  CodMun   MEDIUMINT UNSIGNED,
  AbrevPai VARCHAR(3),
  AbrevDep VARCHAR(3),
  NomMun   VARCHAR(100) NOT NULL,
  NorMun   VARCHAR(100) NOT NULL 
) ENGINE = InnoDB;

In [ ]:
LOAD DATA
LOCAL INFILE '/home/alvaro/Documents/Cursos/Big_Data/Datos/EN_MUNICIPIO.csv'
INTO TABLE EN_MUNICIPIO
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

In [ ]:
ALTER TABLE EN_DEPARTAMENTO
MODIFY COLUMN NomDep VARCHAR(100);

In [ ]:
INSERT INTO MUNICIPIO(NomMun, IdDep)
SELECT M.NomMun, D.IdDep
FROM EN_DEPARTAMENTO AS P, DEPARTAMENTO AS D,  EN_MUNICIPIO AS M
WHERE P.NomDep = D.NomDep AND M.AbrevDep = P.Abrev_Dep;

## CREANDO VISTAS

CREATE [OR REPLACE] VIEW nombre_vista [lista_columnas]
AS consulta

Primero verificamos la consulta de origen

In [ ]:
SELECT D.IdDep, D.NomDep AS Departamento, P.NomPai AS Pais
FROM PAIS AS P 
INNER JOIN DEPARTAMENTO AS D
ON P.IdPai = D.IdPai
WHERE D.NomDep <> "No Aplica";

### Crea la vista

In [ ]:
CREATE OR REPLACE VIEW vDEPARTAMENTO
AS 
SELECT D.IdDep AS IdDep, D.NomDep AS Departamento, P.NomPai AS Pais
FROM PAIS AS P 
INNER JOIN DEPARTAMENTO AS D
ON P.IdPai = D.IdPai
WHERE D.NomDep <> "No Aplica";

### USAR LA VISTA

In [ ]:
SELECT Pais, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
GROUP BY Pais;


In [ ]:
SELECT Pais, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
GROUP BY Pais
HAVING Frecuencia > 12
ORDER BY FRECUENCIA DESC;

### Verificación Colombia

In [ ]:
SELECT Departamento, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
WHERE Pais ="Colombia"
GROUP BY Departamento
ORDER BY FRECUENCIA DESC;

In [ ]:
SELECT * FROM vDEPARTAMENTO
ORDER BY Pais ASC, Departamento ASC;